<a href="https://colab.research.google.com/github/demirhankoc/airline_complaints/blob/main/airline_complaints_sarima_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Anyone who travels by air knows that occasional problems are inevitable. Flights can be delayed or cancelled due to weather conditions, mechanical problems, or labor strikes, and baggage can be lost, delayed, damaged, or pilfered. Given that many airlines are now charging for bags, issues with baggage are particularly annoying. Baggage problems can have a serious impact on customer loyalty, and can be costly to the airlines (airlines often have to deliver bags).
\
\
Air carriers report flight delays, cancellations, overbookings, late arrivals, baggage complaints, and other operating statistics to the U.S. government, which compiles the data and reports it to the public.
![Image](https://storage.googleapis.com/kaggle-datasets-images/3791937/6563141/560fa866709b621b31f2707881f50016/dataset-cover.jpg)

# Preparing Data

### Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline

# Load specific forecasting tools
from statsmodels.tsa.statespace.sarimax import SARIMAX

#from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult      # for ETS Plots
!pip install pmdarima
from pmdarima import auto_arima                              # for determining ARIMA orders

# Load specific evaluation tools
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

### Loading the dataset

In [ ]:
df = pd.read_csv('../input/airline-baggage-complaints-time-series-dataset/baggagecomplaints.csv',index_col='Date',parse_dates=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['Airline'].value_counts()

### Feature engineering

To adjust for company's size we calculate the rate of baggage complaints

In [ ]:
df['Baggage %'] = 100* df['Baggage']/df['Enplaned']

In [ ]:
df

### Creating dataframes for each company, setting index frequency and checking datasets

In [ ]:
grouped = df.groupby(df.Airline)
dfAmerican = grouped.get_group("American Eagle")
dfAmerican

In [ ]:
dfHawaiian = grouped.get_group("Hawaiian")
dfHawaiian

In [ ]:
dfUnited = grouped.get_group("United")
dfUnited

In [ ]:
dfAmerican.index.freq = 'MS' #Month Start frequency
print(len(dfAmerican))
print(dfAmerican.head())

In [ ]:
dfHawaiian.index.freq = 'MS'
print(len(dfHawaiian))
print(dfHawaiian.head())

In [ ]:
dfUnited.index.freq = 'MS'
print(len(dfUnited))
print(dfUnited.head())

# EDA

We start by exploring baggage complaints over time.

In [ ]:
fig = px.line(df, y="Baggage", color="Airline", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)
fig.show()

United Airlines has the most complaints about mishandled baggage in almost all of the months in the data set; Hawaiian Airlines has the fewest number of complaints in all months. Do we conclude, then, that United Airlines has the “worst record” for mishandled baggage and Hawaiian, the best?

In [ ]:
newframe = df.groupby('Airline').describe()[['Scheduled', 'Enplaned']]
newframe = newframe.drop(['count','std','min','25%','50%','75%','max'], axis = 1, level = 1)
newframe = newframe.round(decimals=2)
newframe

United Airlines is a much bigger airline, which shows the average number of scheduled flights and enplaned passengers by airline. United handles more than three times the number of passengers than American Eagle on average, and almost eight times more than Hawaiian. Thus, United has more opportunities to mishandle luggage because it handles more luggage – it’s simply a much bigger airline.

So we compare the records of the three airlines using Baggage %.

In [ ]:
newframe = df.groupby('Airline').describe()['Baggage %']
newframe = newframe.drop(['count','std','min','25%','50%','75%','max'], axis = 1)
newframe = newframe.round(decimals=3)
newframe

We see that American Eagle has the highest rate of baggage complaints when adjusted for number of enplaned passengers.

Plotting the Baggage % on a time series plot allows us to see changes over time.

In [ ]:
fig = px.line(df, y="Baggage %", color="Airline", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)
fig.show()

So we see that baggage complaint rates from American Eagle and United passengers increased through 2006 and began declining thereafter.

The time series for Hawaiian passengers is relatively flat compared to American Eagle and United, so it’s difficult to detect a pattern over time.

In [ ]:
fig = px.line(df, y="Baggage %", color="Airline", facet_col="Airline", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks=None,
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title='',
    matches=None,
    visible=False
)
fig.show()

We isolated the data for the Hawaiian flights. Complaint rates for Hawaiian passengers began to drop in the summer of 2008 until fall of 2010, after which the rate of complaints returned to historical levels.

The pattern of spikes and dips indicates that changes in the rate of baggage complaints may have a seasonal component.

In [ ]:
Baggage_mean = df.groupby(["Airline","Month"]).mean(['Baggage %'])
Baggage_mean = Baggage_mean.reset_index()
Baggage_mean

In [ ]:
fig = px.line(Baggage_mean, x='Month', y='Baggage %', color="Airline", facet_col="Airline", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks=None,
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title='',
    matches=None,
    visible=False
)
fig.update_xaxes(
    dtick = 1
)
fig.show()

We plotted the average Baggage % by month for the three airlines to investigate this further.
\
Average rates are highest in December and January and in the summer months, and lowest in the spring and late fall. Interestingly, all three airlines seem to follow the same general pattern.

### ETS Decomposition

In [ ]:
Baggage_mean = df.groupby(["Date"]).mean(['Baggage'])
Baggage_mean = Baggage_mean.drop(['Month','Year','Scheduled','Cancelled','Enplaned','Baggage %'], axis = 1)
Baggage_mean

In [ ]:
def plot_seasonal_decompose(result:DecomposeResult, dates:pd.Series=None, title:str="Seasonal Decomposition"):
    x_values = dates if dates is not None else np.arange(len(result.observed))
    cols = px.colors.qualitative.Pastel
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.observed, mode="lines", name='Observed', marker=dict(color=cols[0])),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.trend, mode="lines", name='Trend', marker=dict(color=cols[1])),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.seasonal, mode="lines", name='Seasonal', marker=dict(color=cols[2])),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=x_values, y=result.resid, mode="lines", name='Residual', marker=dict(color=cols[3])),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title=f'<b>{title}</b>', margin={'t':100}, title_x=0.5, showlegend=False
        )
    )

In [ ]:
decomposition = seasonal_decompose(Baggage_mean['Baggage'], model='additive', period=12)
fig = plot_seasonal_decompose(decomposition, dates=Baggage_mean.index)

fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)

fig.show()

There's seasonality considering the average number of lost baggage in all airlines.

# SARIMA Model

### Obtaining recommended orders

In [ ]:
auto_arima(Baggage_mean['Baggage'], seasonal=True, m=12).summary()
# m=12 because we have Monthly data

**We might use the ARIMA Order (3, 1, 2) combined with the Seasonal Order (1, 0, [1], 12) as we have the lowest AIC value considering those orders.**

### Splitting the data into train and test sets

**One year (i.e. 12 records) for testing**

In [ ]:
train = Baggage_mean.iloc[:len(Baggage_mean)-12]
test = Baggage_mean.iloc[len(Baggage_mean)-12:]

### Fit a SARIMAX(3, 1, 2)(1, 0, [1], 12) model to the training set

In [ ]:
model = SARIMAX(train['Baggage'], order=(3, 1, 2), seasonal_order=(1, 0, [1], 12))
results = model.fit()
results.summary()

### Obtaining predicted values

In [ ]:
start = len(train)
end = len(train) + len(test) - 1

In [ ]:
predictions = results.predict(start, end, typ='levels').rename('SARIMA Test Predictions')

### Plotting Predictions X Known Values

In [ ]:
cols = px.colors.qualitative.Pastel
fig = go.Figure()
fig.add_trace(go.Scatter(y=test['Baggage'],
                    mode='lines',
                    name='Real Values',
                    marker=dict(color=cols[0])
                        ))
fig.add_trace(go.Scatter(y=predictions,
                    mode='lines',
                    name='SARIMA Test Predictions',
                    marker=dict(color=cols[1])
                        ))
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)
fig.update_xaxes(
    dtick = 1
)
fig.show()

### Evaluating the Model using RMSE

In [ ]:
error = rmse(test['Baggage'], predictions)
print(f'RMSE Error: {error}')

In [ ]:
Baggage_mean.describe()

Comparision with the Standard Deviation and Mean shows us that the RMSE Error is insignificant.

### Retraining the model on the full data and forecasting one year into the future

In [ ]:
model = SARIMAX(Baggage_mean['Baggage'], order=(3, 1, 2), seasonal_order=(1, 0, [1], 12))
results = model.fit()
forecast = results.predict(start=len(Baggage_mean), end=len(Baggage_mean)+11, type='levels').rename('SARIMA Forecast')

### Plotting the forecasted values alongside the original data

In [ ]:
cols = px.colors.qualitative.Pastel
fig = go.Figure()
fig.add_trace(go.Scatter(x=Baggage_mean.index, y=Baggage_mean['Baggage'],
                    mode='lines',
                    name='Real Values',
                    marker=dict(color=cols[0])
                        ))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast,
                    mode='lines',
                    name='SARIMA Test Predictions',
                    marker=dict(color=cols[1])
                        ))
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)

fig.show()

In [ ]:
forecast

### Zooming In - 2009 to 2011 and One-Year Predictions

In [ ]:
cols = px.colors.qualitative.Pastel
fig = go.Figure()
fig.add_trace(go.Scatter(x=Baggage_mean.index[-48:], y=Baggage_mean['Baggage'][-48:],
                    mode='lines',
                    name='Real Values',
                    marker=dict(color=cols[0])
                        ))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast,
                    mode='lines',
                    name='SARIMA Test Predictions',
                    marker=dict(color=cols[1])
                        ))
fig.update_layout(plot_bgcolor='white')
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title=''
)

fig.show()

# Conclusions

* United Airlines is a much bigger airline, which shows the average number of scheduled flights and enplaned passengers by airline. United handles more than three times the number of passengers than American Eagle on average, and almost eight times more than Hawaiian. Thus, United has more opportunities to mishandle luggage because it handles more luggage – it’s simply a much bigger airline.
* Complaint rates for Hawaiian passengers began to drop in the summer of 2008 until fall of 2010, after which the rate of complaints returned to historical levels.
* Average rates are highest in December and January and in the summer months, and lowest in the spring and late fall. Interestingly, all three airlines seem to follow the same general pattern.
* The data set provided did not have information on flight destinations, so we are not able to investigate whether destination is related to baggage issues.
* To provide a fair comparison of performance across airlines, it’s best to standardize for differences in volume. It is also important to pay close attention to units of measurement.
* American Eagle has the highest rate of baggage complaints when adjusted for number of enplaned passengers.
* Baggage complaint rates from American Eagle and United passengers increased through 2006 and began declining thereafter.
* Managers at American Eagle should note the relatively high rate of baggage complaints and consider costs and benefits of improvements.
* All three airlines should anticipate high rates of baggage complaints in January, February and the summer months and should plan accordingly.
* If we are interested in studying baggage complaints for different destinations, additional data are required.
